<a href="https://colab.research.google.com/github/athardolphin/projections/blob/main/HOM(vA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program is the copy of the notebook RSOforSPCAD03, here I am going to implement the same formullas for the fine tuning of RSO for Punjab.


this program aucceaafuly computes the results from this page of EPSG

https://epsg.io/9812-method Hotine Oblique Mercator (variant A)


In [29]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]
import numpy as np
import math

sin,cos,sqrt,pi = np.sin, np.cos,np.sqrt,np.pi
asin,  atan2, tan = np.arcsin,np.arctan2, np.tan

sin, cos, sqrt, log , exp= np.sin, np.cos, np.sqrt, np.log, np.exp
asin, atan2, atan, tan, copysign = np.arcsin, np.arctan2, np.arctan, np.tan, np.copysign
rad, deg = np.radians, np.degrees

In [30]:
def rad_to_dms(rad):
    deg_total = np.degrees(rad)
    degrees = int(deg_total)
    minutes_total = abs(deg_total - degrees) * 60
    minutes = int(minutes_total)
    seconds = (minutes_total - minutes) * 60
    return degrees, minutes, int(seconds)

In [31]:
def basic(Pc, Lc, Ac, k0, e, a):
    e2 = e**2

    # Compute B
    cos_phi0_4 = cos(Pc)**4
    B = sqrt(1 + (e2 * cos_phi0_4) / (1 - e2))

    # Compute A
    sqrt_1_e2 = sqrt(1 - e2)
    denom = 1 - e2 * sin(Pc)**2
    A = (a * B * k0 * sqrt_1_e2) / denom

    # Compute t0 (inline)
    top = tan(pi / 4 - Pc / 2)
    bottom = ((1 - e * sin(Pc)) / (1 + e * sin(Pc)))**(e / 2)
    t0 = top / bottom

    # Compute D
    D = (B * sqrt_1_e2) / (cos(Pc) * sqrt(1 - e2 * sin(Pc)**2))

    # Compute H (same as E)
    H = (D + sqrt(D**2 - 1)) * t0**B

    # Compute F and G
    F = D + sqrt(D**2 - 1)
    G = (F - 1 / F) / 2

    # Compute gamma_0
    g0 =  asin(sin(Ac) / D)
    # print((G * tan(g0)),"Gtan")
    # Compute lam0
    lam0 = Lc - asin(G * tan(g0)) / B

    # Compute uc
    Dsq = D * D
    sqrt_term = sqrt(Dsq - 1)

    atan_term = atan2(sqrt_term, cos(Ac))
    sign_latc = math.copysign(1.0, Pc) if Pc != 0 else 0.0
    uc = (A / B) * atan_term * sign_latc
    vc = 0
    # Print all variables
    # print("B =", B)
    # print("A =", A)
    # print("t0 =", t0)
    # print("D =", D)
    # print("H =", H)
    # print("F =", F)
    # print("G =", G)
    # print("gamma_0 [rad] =", g0)
    # print("gamma_0 [deg] =", np.degrees(g0))
    # print("lam0 [rad] =", lam0)
    # print("lam0 [deg] =", np.degrees(lam0))
    # print("uc =", uc)

    return B, A, t0, D, H, F, G, g0, lam0, uc


In [35]:
def fConst(x0,y0,e,A,B,H, P, L,g0,L0):
    t = tan(pi / 4 - P / 2) / ((1 - e * sin(P)) / (1 + e * sin(P))) ** (e / 2)
    Q = H / (t ** B)
    S = (Q - 1 / Q) / 2
    T = (Q + 1 / Q) / 2
    V = sin(B * (L - L0))
    U = (-V * cos(g0) + S * sin(g0)) / T
    v = (A * log((1 - U) / (1 + U))) / (2 * B)


    u = (A / B) * atan2(S * cos(g0) + V * sin(g0) , cos(B * (L - L0)))
    k = (A * cos(B * u / A) * sqrt(1 - e2 * sin(P)**2)) / (a * cos(P) * cos(B * (L - L0)))
    # x and y (Easting and Northing)
    x = v * cos(g0) + u * sin(g0) + x0
    y = u * cos(g0) - v * sin(g0) + y0
    # print('t', t)
    # print('Q', Q)
    # print('S', S)
    # print('T', T)
    # print('V', V)
    # print('U', U)
    # print('v', v)
    # print('u', u)
    # print('k', k)
    print('x', x)
    print('y', y)
    return x,y,t,Q,S,T,V,U

In [33]:
def rConst(E,N,Ec,Nc,g0, L0, A, B, H, e):
    v_ = (E - Ec) * cos(g0) - (N - Nc) * sin(g0)
    u_ = (N - Nc) * cos(g0) + (E - Ec) * sin(g0)

    Q_ = exp(-B * v_ / A)
    S_ = 0.5 * (Q_ - 1 / Q_)
    T_ = 0.5 * (Q_ + 1 / Q_)
    V_ = sin(B * u_ / A)
    U_ = (V_ * cos(g0) + S_ *sin(g0)) / T_

    # Step 3: Compute t’
    t_ = (H / sqrt((1 + U_) / (1 - U_)))**(1 / B)

    # Step 4: Compute conformal latitude chi
    chi = pi / 2 - 2 * atan(t_)

    # Step 5: Compute geographic latitude using series expansion
    e2 = e * e
    e4 = e2 * e2
    e6 = e4 * e2
    e8 = e4 * e4

    lat = chi \
        + sin(2 * chi) * (e2 / 2 + 5 * e4 / 24 + e6 / 12 + 13 * e8 / 360) \
        + sin(4 * chi) * (7 * e4 / 48 + 29 * e6 / 240 + 811 * e8 / 11520) \
        + sin(6 * chi) * (7 * e6 / 120 + 81 * e8 / 1120) \
        + sin(8 * chi) * (4279 * e8 / 161280)

    # Step 6: Compute longitude

    lon = L0 - atan2(S_ * cos(g0) - V_ * sin(g0) , cos(B * u_ / A)) / B

    # print("L0  =", deg(L0))
    # print("v'  =", v_)
    # print("u'  =", u_)
    # print("Q'  =", Q_)
    # print("S'  =", S_)
    # print("T'  =", T_)
    # print("V'  =", V_)
    # print("U'  =", U_)
    # print("t'  =", t_)
    # print("chi =", chi)
    print("lat =", deg(lat),rad_to_dms(lat))
    print("lon =", deg(lon),rad_to_dms(lon))

    return lat, lon


In [36]:
# Given values from epsg web and doc
a = 6377298.556
e2 = 0.006637847
# Compute eccentricity
e = e2**(1/2)
Pc = np.radians(4)
Lc = np.radians(115)
P  = 0.094025313
L  = 2.021187362
Ac = 0.930536611
k0 = 0.99984
Ec = 0
Nc = 0

B, A, t0, D, H, F, G, g0, L0, uc =  basic(Pc,Lc, Ac, k0, e, a)
E,N,t,Q,S,T,V,U = fConst(Ec,Nc,e,A,B,H, P, L,g0,L0)
lat, lon = rConst(E,N,Ec,Nc,g0, L0, A, B, H, e)

x 679245.7339646654
y 596562.7859130147
lat = 5.3872536022320565 (5, 23, 14)
lon = 115.8055054477805 (115, 48, 19)
